In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.datasets import load_sample_image

In [ ]:
china = load_sample_image("china.jpg")/255
flower = load_sample_image("flower.jpg")/255

In [ ]:
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

In [ ]:
filters = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)
filters[:, 3, :, 0] = 1
filters[3, :, :, 1] = 1

In [ ]:
outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")

In [ ]:
plt.imshow(outputs[0, :, :, 1], cmap="gray")

In [ ]:
plt.imshow(outputs[0, :, :, 0], cmap="gray")

---
# MNIST

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
def preprocess_mnist(data):
    data = tf.expand_dims(data, 3)
    return data

In [4]:
mnist = keras.datasets.fashion_mnist.load_data()

from sklearn.model_selection import train_test_split

(x_train_full, y_train_full), (x_test, y_test) = mnist
x_train, y_train = x_train_full[: 50000], y_train_full[: 50000]
x_val, y_val = x_train_full[50000: ], y_train_full[50000: ]

batch_size = 256

train_set = tf.data.Dataset.from_tensor_slices((preprocess_mnist(x_train), y_train)).shuffle(len(x_train)).batch(batch_size).prefetch(1)
valid_set = tf.data.Dataset.from_tensor_slices((preprocess_mnist(x_val), y_val)).batch(batch_size).prefetch(1)
test_set = tf.data.Dataset.from_tensor_slices((preprocess_mnist(x_test), y_test)).batch(batch_size).prefetch(1)

In [5]:
normalization = keras.layers.experimental.preprocessing.Normalization()
normalization.adapt(x_train)

In [6]:
preprocess_mnist(x_train)

<tf.Tensor: shape=(50000, 28, 28, 1), dtype=uint8, numpy=
array([[[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]]],


       [[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0]

In [7]:
keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.Input(shape=[28, 28, 1]),
    normalization,
    keras.layers.Conv2D(64, 7, activation="elu", kernel_initializer="he_normal",
                        padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(128, 3, activation="elu", kernel_initializer="he_normal",
                        padding="same"),
    keras.layers.Conv2D(128, 3, activation="elu", kernel_initializer="he_normal",
                        padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(256, 3, activation="elu", kernel_initializer="he_normal",
                        padding="same"),
    keras.layers.Conv2D(256, 3, activation="elu", kernel_initializer="he_normal",
                        padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [9]:
import os
from datetime import datetime
logs = os.path.join(os.curdir, "my_msnit_logs",
                    "run_" + datetime.now().strftime("%Y%m%d_%H%M%S"))

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir = logs)
early_cb = tf.keras.callbacks.EarlyStopping(patience=10)


model.fit(train_set, epochs=100, validation_data=valid_set, callbacks=[tensorboard_cb, early_cb])

Epoch 1/100
196/196 [==============================] - 16s 63ms/step - loss: 2.4429 - accuracy: 0.5218 - val_loss: 0.4243 - val_accuracy: 0.8536
Epoch 2/100
196/196 [==============================] - 11s 55ms/step - loss: 0.5654 - accuracy: 0.8167 - val_loss: 0.3942 - val_accuracy: 0.8592
Epoch 3/100
196/196 [==============================] - 11s 54ms/step - loss: 0.4439 - accuracy: 0.8535 - val_loss: 0.3559 - val_accuracy: 0.8714
Epoch 4/100
196/196 [==============================] - 11s 54ms/step - loss: 0.3669 - accuracy: 0.8754 - val_loss: 0.3108 - val_accuracy: 0.8910
Epoch 5/100
196/196 [==============================] - 11s 54ms/step - loss: 0.3290 - accuracy: 0.8882 - val_loss: 0.2974 - val_accuracy: 0.8978
Epoch 6/100
196/196 [==============================] - 11s 54ms/step - loss: 0.2850 - accuracy: 0.9018 - val_loss: 0.4225 - val_accuracy: 0.8675
Epoch 7/100
196/196 [==============================] - 11s 54ms/step - loss: 0.2705 - accuracy: 0.9083 - val_loss: 0.2909 - val_ac

In [10]:
for img, label in train_set:
    print(model.predict(img))

[[8.9566359e-29 1.0000000e+00 2.9238023e-35 ... 3.4355635e-29
  2.5510836e-33 9.6363784e-29]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [8.0067678e-24 1.0034302e-25 5.3394048e-29 ... 2.9918977e-25
  1.0000000e+00 1.3563981e-22]
 ...
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.7533157e-02 8.6881673e-01 4.8525434e-04 ... 9.7899056e-06
  2.7260427e-05 4.2103293e-06]
 [2.5706519e-32 1.0000000e+00 2.3775475e-38 ... 1.1294150e-31
  1.4630571e-36 1.6080906e-31]]
[[9.9998903e-01 2.4915327e-18 6.1100631e-11 ... 7.4493822e-27
  5.9897001e-19 2.6684315e-24]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.2014461e-18
  0.0000000e+00 1.0000000e+00]
 [2.2480090e-33 9.1545444e-25 1.5392591e-36 ... 9.9999928e-01
  9.0574136e-27 7.3021044e-07]
 ...
 [9.9869341e-01 3.3956672e-16 2.6808396e-09 ... 8.8302379e-22
  7.5364344e-16 9.2960173e-20]
 [1.9136352e-10 2.2058291e-11 2.6054502e-11 ... 1.3671342e-

[[2.3030002e-15 3.9038564e-11 8.1634081e-16 ... 9.8373759e-01
  1.2916784e-12 1.5310522e-02]
 [9.9999654e-01 2.6903416e-19 5.1489260e-12 ... 4.8767604e-29
  6.0918466e-20 2.2284245e-26]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [3.9390980e-05 4.3000134e-07 8.5662510e-03 ... 1.4316088e-11
  9.5631245e-09 4.1776797e-11]
 [0.0000000e+00 1.8212543e-30 0.0000000e+00 ... 2.0286684e-20
  1.3168039e-30 8.9274057e-21]
 [0.0000000e+00 7.1220753e-31 0.0000000e+00 ... 5.5316290e-10
  1.6817253e-32 1.0000000e+00]]
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.4755862e-26
  0.0000000e+00 1.0000000e+00]
 [5.7137208e-03 8.1533653e-06 1.1220639e-02 ... 5.0390742e-07
  1.3638478e-05 1.1033132e-07]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.0281015e-19
  0.0000000e+00 1.0000000e+00]
 ...
 [2.0039614e-09 1.8636750e-11 1.9887851e-04 ... 6.3470402e-20
  1.2243977e-15 1.6860910e-18]
 [4.9011401e-34 1.0000000e+00 0.0000000e+00 ... 2.7057851e-

[[2.57740497e-22 1.83145091e-14 1.00616976e-23 ... 6.48067726e-05
  1.07798308e-12 3.51307355e-02]
 [1.14534901e-12 1.22078129e-12 2.21798034e-14 ... 3.04019266e-12
  1.00000000e+00 1.78949716e-10]
 [1.44625903e-06 1.21204324e-09 6.65595189e-06 ... 3.53691346e-18
  3.69425492e-13 1.27642923e-18]
 ...
 [9.99999881e-01 1.46791665e-19 7.73011429e-12 ... 3.12083161e-30
  8.78413869e-21 6.66419447e-27]
 [0.00000000e+00 7.18700871e-37 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 6.62253516e-11]
 [0.00000000e+00 1.05688084e-35 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 1.48278629e-11]]
[[4.4607063e-05 5.7297922e-10 1.1373666e-05 ... 2.5370815e-14
  1.0850979e-10 6.1371413e-14]
 [1.5037783e-32 5.7442276e-23 5.6862349e-38 ... 9.2640923e-13
  1.4587758e-23 9.0453259e-14]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 1.1876772e-33 0.0000000e+00 ... 1.1998995e-11
  8.9448545e-37 1.0000000e+00]
 [0.0000000e+00 6.37063

[[5.39292932e-01 6.06373360e-05 9.71177686e-03 ... 3.99675700e-05
  1.47557628e-04 6.24109234e-05]
 [8.39140533e-24 1.00000000e+00 1.77609086e-27 ... 3.88759647e-24
  2.30848473e-27 8.42138401e-25]
 [1.73698704e-20 7.73776410e-21 6.27161313e-21 ... 3.79802997e-18
  1.00000000e+00 5.10123051e-16]
 ...
 [9.98215973e-01 4.05902013e-11 2.48168772e-06 ... 1.17245443e-15
  1.30618242e-11 3.71594065e-14]
 [6.85778820e-32 1.04319856e-22 6.11534635e-35 ... 6.12073472e-06
  1.98983466e-21 9.99993920e-01]
 [2.46587795e-09 1.40217470e-20 9.99998927e-01 ... 3.02984242e-25
  8.85847736e-18 1.39780528e-21]]
[[8.2253158e-31 5.7641722e-21 1.2967404e-32 ... 2.5432817e-05
  4.6949476e-23 9.9997461e-01]
 [9.9879515e-01 5.9596023e-11 3.0635045e-05 ... 7.4568969e-16
  2.4057163e-11 1.4512061e-14]
 [0.0000000e+00 7.7684328e-29 0.0000000e+00 ... 6.3717769e-09
  2.5842397e-32 1.0000000e+00]
 ...
 [5.1195330e-32 1.0000000e+00 2.3230388e-38 ... 8.9384684e-31
  6.3682177e-35 1.8753925e-31]
 [0.0000000e+00 1.00000

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.8443832e-14
  0.0000000e+00 1.0000000e+00]
 [7.8631128e-06 2.9189133e-07 5.0432618e-07 ... 2.4686246e-11
  1.5355824e-09 2.4485308e-11]
 [4.3099906e-11 2.3359258e-29 1.1011001e-15 ... 1.2198069e-37
  3.5130033e-27 5.7016186e-36]
 ...
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.8560184e-01 3.8884111e-05 5.5845724e-03 ... 2.2303113e-05
  4.0996359e-03 2.3691812e-06]
 [4.2230982e-29 1.2456261e-19 4.1198557e-31 ... 2.2099607e-06
  9.4692074e-21 9.9999762e-01]]
[[1.1305805e-28 7.8289147e-21 5.0167621e-31 ... 9.9999607e-01
  2.8092798e-23 3.8537455e-06]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.4916339e-27
  0.0000000e+00 3.2724171e-28]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.0836807e-20
  0.0000000e+00 1.0000000e+00]
 ...
 [2.1787557e-25 1.0000000e+00 8.1670925e-30 ... 4.1863558e-24
  4.5675296e-26 8.2015802e-25]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+

...

[[3.2722633e-04 1.4474020e-07 9.8382437e-01 ... 1.0193359e-08
  8.4372567e-07 2.3484771e-08]
 [2.3744794e-06 2.2378472e-11 2.1629230e-11 ... 6.6960686e-15
  5.8919905e-13 9.2894095e-15]
 [3.6742451e-06 3.4656901e-07 7.8728732e-08 ... 9.6322783e-11
  1.0016487e-09 6.9193949e-11]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 1.9382827e-11]
 [5.1713994e-11 2.7387020e-13 3.1599914e-06 ... 1.2129309e-22
  9.1276345e-18 5.1395034e-21]
 [7.2227016e-17 2.4897547e-18 1.1611180e-23 ... 8.3387069e-20
  1.0000000e+00 1.0243885e-18]]
[[6.59824535e-02 5.31497779e-09 4.00210083e-05 ... 1.59427271e-09
  1.44193137e-07 4.03573974e-09]
 [0.00000000e+00 3.89818791e-30 0.00000000e+00 ... 5.09568332e-10
  3.17025133e-33 1.00000000e+00]
 [9.98356164e-01 1.49285917e-18 8.31373512e-12 ... 6.89154310e-26
  7.62101936e-21 4.66508437e-24]
 ...
 [7.50884938e-05 7.84666767e-08 9.81659591e-01 ... 6.82312054e-11
  8.45356354e-08 4.07875206e-10]
 [0.00000000e+00 0.00000000e+00 0.0

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.2677240e-26
  2.1895476e-38 2.4772000e-27]
 [4.9862056e-04 1.4614678e-05 5.3460771e-01 ... 2.7228011e-09
  1.6648643e-07 5.9744192e-09]
 [3.7351612e-15 2.2257164e-16 4.7440095e-19 ... 5.1254763e-16
  1.0000000e+00 6.1331551e-15]
 ...
 [2.4021434e-19 1.1346259e-14 2.6908452e-21 ... 9.4344714e-06
  8.2686511e-13 3.4616488e-07]
 [5.6064415e-09 4.0372213e-24 2.6361461e-12 ... 3.0801361e-29
  7.5438826e-22 8.5894941e-29]
 [6.4647761e-13 3.9930922e-14 2.2773158e-16 ... 1.8299317e-15
  1.0000000e+00 5.3961958e-14]]
[[9.43903376e-12 3.20432892e-27 5.28740497e-14 ... 5.44916284e-36
  4.42982284e-26 2.12923224e-35]
 [1.64053533e-21 1.00000000e+00 1.74963610e-27 ... 1.08585224e-26
  3.17502956e-27 9.52379876e-26]
 [9.99911308e-01 4.27307996e-14 2.06815045e-08 ... 1.79938360e-21
  1.13210248e-14 2.67633512e-19]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 5.53039949e-12]
 [3.85706517e-04 2.06342406e-07 9.6

[[1.34175290e-28 1.00000000e+00 7.93465790e-36 ... 1.12211445e-31
  4.43646767e-34 2.17272334e-31]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 2.98994358e-12]
 [1.26783192e-23 1.00000000e+00 2.40350472e-27 ... 3.93040985e-23
  2.42262844e-23 3.39863271e-23]
 ...
 [3.51838647e-09 2.39074177e-10 1.44581812e-11 ... 3.26169612e-11
  1.00000000e+00 7.40411343e-10]
 [0.00000000e+00 1.25111729e-30 0.00000000e+00 ... 6.78295677e-08
  5.07810327e-32 9.99999881e-01]
 [0.00000000e+00 2.90917554e-25 0.00000000e+00 ... 8.87293865e-08
  1.22805825e-27 9.99999881e-01]]
[[0.00000000e+00 1.29663811e-27 0.00000000e+00 ... 6.92644690e-18
  1.88873392e-29 4.37499031e-19]
 [1.15169041e-06 1.42244078e-10 7.42456632e-06 ... 1.39400368e-16
  2.67229679e-13 5.25369208e-17]
 [1.07814983e-12 5.91977232e-14 5.55082883e-15 ... 7.59613838e-25
  7.32347768e-22 7.56912186e-25]
 ...
 [9.94957626e-01 2.71456403e-12 3.83827768e-07 ... 1.88454216e-16
  2.89170753e-11 6.58877622e-15]

[[7.9023092e-17 2.5667709e-14 3.4675318e-18 ... 9.9922800e-01
  3.3786318e-14 7.7007303e-04]
 [1.9456747e-08 5.8633709e-10 4.7601649e-04 ... 3.7369307e-17
  1.5775467e-12 1.8146758e-15]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.8128769e-27
  0.0000000e+00 3.6292046e-29]
 ...
 [2.2826330e-21 3.7579702e-23 3.6146315e-27 ... 1.1709839e-23
  1.0000000e+00 9.6453500e-22]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [9.9561834e-01 2.1249066e-12 1.2343109e-07 ... 9.4488883e-18
  4.1436299e-13 3.8336656e-16]]
[[9.9262692e-08 2.3561795e-09 1.6220490e-11 ... 1.0314835e-15
  2.3802580e-14 2.8275467e-15]
 [1.4849576e-05 6.6981267e-04 1.2520969e-05 ... 5.7843727e-01
  3.5868474e-05 1.8829390e-02]
 [9.4328316e-05 2.6466975e-07 1.8461270e-03 ... 1.3088595e-10
  6.6370078e-09 9.4407843e-11]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 2.1475376e-12]
 [3.4306883e-31 1.0000000e+00 1.1040534e-36 ... 6.6081628e-

[[3.14852476e-26 1.26852991e-16 1.04111256e-27 ... 1.21744554e-02
  2.56460070e-18 9.87823248e-01]
 [0.00000000e+00 1.78906911e-30 0.00000000e+00 ... 5.04398452e-19
  1.19920025e-32 5.10814967e-21]
 [2.94975333e-09 2.12789291e-12 8.89733708e-13 ... 1.11307936e-18
  3.04925746e-16 1.55974345e-18]
 ...
 [1.54496207e-19 3.09744404e-22 3.70371335e-26 ... 1.27875216e-21
  1.00000000e+00 2.10522777e-21]
 [8.95985425e-01 1.25392563e-09 3.47316964e-05 ... 2.18213937e-11
  1.59372107e-08 2.01870812e-10]
 [0.00000000e+00 1.20167012e-27 0.00000000e+00 ... 1.20699383e-06
  1.27542228e-30 9.99998808e-01]]
[[9.1165635e-18 1.0000000e+00 1.3498749e-20 ... 1.2651296e-18
  6.0938463e-19 2.1932805e-19]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [9.9982893e-01 3.0094723e-16 2.5000653e-09 ... 9.0703353e-24
  3.9301991e-17 1.1269516e-20]
 [1.5021494e-02 8.74955